# Dataframe running SQL query


### Create a SparkSession and read the a stock price dataset CSV

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession

In [2]:
spark1 = SparkSession.builder.appName('SQL').getOrCreate()

In [3]:
df = spark1.read.csv('Data/appl_stock.csv',inferSchema=True,header=True)

In [4]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



### Create a temporary view

In [5]:
df.createOrReplaceTempView('stock')

### Now run a simple SQL query directly on this view. It returns a DataFrame.

In [6]:
result = spark1.sql("SELECT * FROM stock LIMIT 5")
result

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: int, Adj Close: double]

In [7]:
result.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [8]:
result.show()

+-------------------+----------+----------+------------------+------------------+---------+------------------+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+-------------------+----------+----------+------------------+------------------+---------+------------------+



### Run slightly more complex queries
How many entries in the `Close` field are higher than 500?

In [9]:
count_greater_500 = spark1.sql("SELECT COUNT(Close) FROM stock WHERE Close > 500").show()

+------------+
|count(Close)|
+------------+
|         403|
+------------+



What is the average `Open` values of all the entries where `Volume` is either greater than 120 million or less than 110 million?

In [10]:
avg_1 = spark1.sql("SELECT AVG(Open) FROM stock WHERE Volume > 120000000 OR Volume < 110000000").show()

+------------------+
|         avg(Open)|
+------------------+
|309.12406365290224|
+------------------+



### Read a file (and create dataframe) by directly running a `spark.sql` method on the file
Notice the syntax of `csv.<path->filename.csv>` inside the SQL query

In [13]:
df_sales = spark1.sql("SELECT * FROM csv.`Data/sales_info.csv`")

In [14]:
df_sales.show()

+-------+-------+-----+
|    _c0|    _c1|  _c2|
+-------+-------+-----+
|Company| Person|Sales|
|   GOOG|    Sam|  200|
|   GOOG|Charlie|  120|
|   GOOG|  Frank|  340|
|   MSFT|   Tina|  600|
|   MSFT|    Amy|  124|
|   MSFT|Vanessa|  243|
|     FB|   Carl|  870|
|     FB|  Sarah|  350|
|   APPL|   John|  250|
|   APPL|  Linda|  130|
|   APPL|   Mike|  750|
|   APPL|  Chris|  350|
+-------+-------+-----+

